In [3]:
from scipy.signal import butter, lfilter
import fastdyn_fic_dmf as dmf
import numpy as np
import matplotlib.pyplot as plt
# Fetch default parameters
import tracemalloc
from scipy.io import loadmat
from scipy.stats import zscore, pearsonr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mat73
# Helper functions
def compute_fcd(data, wsize, overlap, isubdiag):
    T, N = data.shape
    win_start = np.arange(0, T - wsize - 1, wsize - overlap)
    nwins = len(win_start)
    fcd = np.zeros((len(isubdiag[0]), nwins))
    #print(fcd.shape)
    #print(data.shape)
    #print((data[win_start[2]:win_start[2] + wsize + 1, :]).shape)
    for i in range(nwins):
        tmp = data[win_start[i]:win_start[i] + wsize + 1, :]
        cormat = np.corrcoef(tmp.T)
        fcd[:, i] = cormat[isubdiag[0],isubdiag[1]]
    return fcd


C = loadmat('./data/DTI_fiber_consensus_HCP.mat')['connectivity'][:200, :200]
C = 0.2*C/np.max(C)
N = C.shape[0]

G_VAL = 3.5
NB_STEPS = 15000
BURNOUT = 5000
SEED_BASE = 1  # Base seed value

# Define the list of LR values you want to test
lr_values = [500, 15000, 20000, 25000]

# Assuming N is defined globally (number of regions)
N = 200  # adjust accordingly if different

In [2]:

def sim_run(G_VAL, LR, SEED, NB_STEPS=50000):
    """
    INPUTS:
    G_VAL: float, global coupling
    LR: array, learning rate (Homogeneous or heterogenos. Decay will be calcualted for each region with this)
    SEED: int, random seed
    OUTPUTS:
    rates_dyn: np.array, dynamic of rates
    rates_inh_dyn: np.array, dynamic of inhibitory rates
    bold_dyn: np.array, dynamic of BOLD signal
    fic_t_dyn: np.array, dynamic of FIC

    """
    
    params = dmf.default_params(C=C)
    fit_res = np.load("./data/fit_res_3-44.npy")
    b = fit_res[0] # First element is the slope
    a = fit_res[1]
    params['G'] = G_VAL
    params['seed'] = SEED
    params['obj_rate'] = 3.44
    DECAY = np.exp(a+np.log(LR)*b)    
    params['lr_vector'] = LR
    params['taoj_vector'] =  DECAY
    params['J'] = 0.75*params['G']*params['C'].sum(axis=0).squeeze() + 1
    params["with_decay"] = True
    params["with_plasticity"] = True
    params['return_bold'] = False
    params["return_fic"] = False
    params["return_rate"] = True
    rates_dyn, rates_inh_dyn, _, fic_t_dyn = dmf.run(params, NB_STEPS)
    return rates_dyn, rates_inh_dyn, fic_t_dyn


import numpy as np

def vectorize_along_axis(axis=0):
    def decorator(func):
        def wrapper(data, *args, **kwargs):
            # if the data is 1D, just call the function directly
            if data.ndim == 1:
                return func(data, *args, **kwargs)
            # otherwise, apply the function along the specified axis
            return np.apply_along_axis(func, axis, data, *args, **kwargs)
        return wrapper
    return decorator

@vectorize_along_axis(axis=0)
def get_autcorr(rates):
    """Get the autocorrelation function from a 1D rates vector."""
    signal = rates - np.mean(rates)
    # Calculate autocorrelation function (full convolution)
    autocorr = np.correlate(signal, signal, mode='full')
    # Normalize: divide by the variance and length of the signal
    autocorr = autocorr / (np.var(signal) * len(signal))
    # Only keep the second half (non-negative lags)
    autocorr = autocorr[len(signal)-1:]
    return autocorr
# create a function that computes and plots the autocorrelation of the average rates
def plot_autocorr(rates, title):
    autocorr = get_autcorr(np.mean(rates, axis=0))
    lags = np.arange(0, len(autocorr))
    plt.plot(lags, autocorr)
    plt.title(title)
    plt.xlabel('Lag')
    plt.ylabel('Autocorrelation')
    plt.show()



In [ ]:
import numpy as np
import os
from joblib import Parallel, delayed



def run_simulation(idx, lr):
    SEED = SEED_BASE + idx
    # Create a homogeneous LR vector for all regions
    LR_VEC = np.ones(N) * lr
    # Run simulation (assuming sim_run returns rates, inhibitory rates and fic_t in that order)
    rates, _, _ = sim_run(G_VAL, LR_VEC, SEED, NB_STEPS)
    # Discard burnout period
    return rates[:, BURNOUT:]

for lr in lr_values:
    # Execute 100 simulation runs in parallel.
    simulations = Parallel(n_jobs=16)(delayed(run_simulation)(idx, lr) for idx in range(100))
    rates_all = np.array(simulations)
    
    # Ensure the output directory exists
    os.makedirs("./Results/homogeneous", exist_ok=True)
    filename = f"./Results/homogeneous/g_{G_VAL}_lr_{lr}.npy"
    np.save(filename, rates_all)
    print(f"Saved homogeneous simulation results for LR = {lr} to {filename}")

Saved homogeneous simulation results for LR = 500 to ./Results/homogeneous/g_3.5_lr_500.npy
Saved homogeneous simulation results for LR = 15000 to ./Results/homogeneous/g_3.5_lr_15000.npy
Saved homogeneous simulation results for LR = 20000 to ./Results/homogeneous/g_3.5_lr_20000.npy
Saved homogeneous simulation results for LR = 25000 to ./Results/homogeneous/g_3.5_lr_25000.npy


## DL approach

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import scipy.signal as signal

############################
# 1) PhaseDiffDataset
############################
class PhaseDiffDataset(Dataset):
    """
    Converts raw 'rates' of shape (N, 200, 50000) into sequences of 20 phase-difference
    matrices, each 200x200. Final shape: (N, 20, 1, 200, 200).
    """
    def __init__(self, rates, num_timepoints=20):
        """
        rates: NumPy array of shape (N=150, 200, 50000).
        num_timepoints: how many equally spaced time points to sample for each simulation.
        """
        super().__init__()
        self.num_timepoints = num_timepoints
        self.phase_diff = self._create_phase_diff_matrices(rates, num_timepoints)

    def _create_phase_diff_matrices(self, rates, num_matrices):
        """
        1) Apply Hilbert transform -> instantaneous phase
        2) Sample 'num_matrices' time points from each simulation
        3) Build 200x200 phase-difference matrix at each time point
        Returns a NumPy array of shape (N, num_matrices, 1, 200, 200).
        """
        N, num_channels, timesteps = rates.shape  # e.g. (150, 200, 50000)

        # Hilbert transform along time axis -> instantaneous phase
        analytic_signal = signal.hilbert(rates, axis=2)    # shape (N, 200, 50000)
        inst_phase = np.angle(analytic_signal)             # same shape

        # Time indices for sampling
        time_indices = np.linspace(0, timesteps - 1, num_matrices, dtype=int)

        all_phase_diff = []
        for i in range(N):
            # Extract the phase for the ith simulation -> shape (200, 50000)
            phase_i = inst_phase[i]

            # Collect phase-diff matrices for the 20 time points
            diff_list = []
            for t in time_indices:
                phase_t = phase_i[:, t]  # shape (200,)
                diff_matrix = phase_t[:, None] - phase_t[None, :]  # (200, 200)
                diff_list.append(diff_matrix[None, ...])          # (1, 200, 200)

            # Shape -> (20, 200, 200) for this simulation
            diff_array = np.concatenate(diff_list, axis=0)
            # Keep for each simulation
            all_phase_diff.append(diff_array[None, ...])   # (1, 20, 200, 200)

        # Combine => (N, 20, 200, 200)
        all_phase_diff = np.concatenate(all_phase_diff, axis=0)
        # Add channel dimension => (N, 20, 1, 200, 200)
        all_phase_diff = all_phase_diff[:, :, None, :, :]
        return all_phase_diff

    def __len__(self):
        return self.phase_diff.shape[0]

    def __getitem__(self, idx):
        # Return a float tensor of shape (20, 1, 200, 200)
        return torch.from_numpy(self.phase_diff[idx]).float()


############################
# 2) ConvLSTM building blocks
############################
class ConvLSTMCell(nn.Module):
    """
    A single ConvLSTM cell for 2D frames.
    Input shape: (B, in_channels, H, W)
    Output: h_next, c_next
    """
    def __init__(self, input_channels, hidden_channels, kernel_size=3, padding=1):
        super().__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        
        self.conv = nn.Conv2d(
            in_channels=input_channels + hidden_channels,
            out_channels=4 * hidden_channels,
            kernel_size=kernel_size,
            padding=padding
        )

    def forward(self, x, h, c):
        # x, h => (B, channels, H, W)
        combined = torch.cat([x, h], dim=1)  # concat along channel axis
        gates = self.conv(combined)         # => (B, 4*hidden_channels, H, W)
        
        # Split into i, f, g, o
        i, f, g, o = torch.split(gates, self.hidden_channels, dim=1)
        
        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        g = torch.tanh(g)
        o = torch.sigmoid(o)
        
        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)
        
        return h_next, c_next

    def init_hidden(self, batch_size, spatial_size):
        height, width = spatial_size
        device = next(self.parameters()).device
        h = torch.zeros(batch_size, self.hidden_channels, height, width, device=device)
        c = torch.zeros(batch_size, self.hidden_channels, height, width, device=device)
        return h, c

class ConvLSTM(nn.Module):
    """
    Single-layer ConvLSTM that processes a sequence of 2D frames:
      Input shape: (B, T, in_channels, H, W)
      Output: 
        if return_sequence=True -> (B, T, hidden_channels, H, W)
        else -> final (h, c)
    """
    def __init__(self, input_channels, hidden_channels, kernel_size=3, padding=1, return_sequence=False):
        super().__init__()
        self.hidden_channels = hidden_channels
        self.cell = ConvLSTMCell(input_channels, hidden_channels, kernel_size, padding)
        self.return_sequence = return_sequence

    def forward(self, x):
        batch_size, timesteps, _, height, width = x.size()
        h, c = self.cell.init_hidden(batch_size, (height, width))
        
        outputs = []
        for t in range(timesteps):
            x_t = x[:, t]          # shape: (B, in_channels, H, W)
            h, c = self.cell(x_t, h, c)
            if self.return_sequence:
                outputs.append(h.unsqueeze(1))  # store each time step's h
        
        if self.return_sequence:
            return torch.cat(outputs, dim=1), (h, c)
        else:
            return h, c

############################
# 3) ConvLSTMAutoEncoder
############################
class ConvLSTMAutoEncoder(nn.Module):
    """
    Shallow autoencoder with:
      - encoder_lstm (no sequence output)
      - decoder_lstm (returns the entire sequence)
      - final conv to map hidden_channels -> input_channels
    """
    def __init__(self, input_channels=1, hidden_channels=8, kernel_size=3, padding=1):
        super().__init__()
        self.encoder_lstm = ConvLSTM(
            input_channels=input_channels,
            hidden_channels=hidden_channels,
            kernel_size=kernel_size,
            padding=padding,
            return_sequence=False
        )
        self.decoder_lstm = ConvLSTM(
            input_channels=hidden_channels,
            hidden_channels=hidden_channels,
            kernel_size=kernel_size,
            padding=padding,
            return_sequence=True
        )
        self.conv_out = nn.Conv2d(hidden_channels, input_channels, kernel_size=1, padding=0)

    def forward(self, x):
        """
        x: (B, T=20, in_channels=1, H=200, W=200)
        Returns reconstruction => same shape (B, T, 1, H, W).
        """
        B, T, C, H, W = x.shape
        
        # --- Encoder ---
        h_enc, c_enc = self.encoder_lstm(x)   # final hidden/cell from encoder
        
        # --- Decoder ---
        # We'll feed zeros at each time step (could use teacher forcing if desired)
        decoder_input = torch.zeros((B, T, self.decoder_lstm.hidden_channels, H, W), 
                                    device=x.device, dtype=x.dtype)
        h_dec, c_dec = (h_enc, c_enc)
        
        outputs = []
        for t in range(T):
            x_t = decoder_input[:, t]
            h_dec, c_dec = self.decoder_lstm.cell(x_t, h_dec, c_dec)
            outputs.append(h_dec.unsqueeze(1))
        
        # shape => (B, T, hidden_channels, H, W)
        outputs = torch.cat(outputs, dim=1)
        
        # map each frame to 1 channel
        recon_frames = []
        for t in range(T):
            out_t = self.conv_out(outputs[:, t])
            recon_frames.append(out_t.unsqueeze(1))
        
        # shape => (B, T, 1, H, W)
        recon_sequence = torch.cat(recon_frames, dim=1)
        return recon_sequence

############################
# 4) Main script
############################
if __name__ == "__main__":
    # Hyperparams
    batch_size = 2
    lr = 1e-3
    num_epochs = 2
    lr_values = [100,200,1000,10000,15000,20000]
    # Load rates data. Pile all the different LR values along the first axis
    rates = np.zeros((len(lr_values)*100, 200, NB_STEPS-BURNOUT))
    for lr in lr_values:
        filename = f"./Results/homogeneous/g_{G_VAL}_lr_{lr}.npy"
        rates_lr = np.load(filename)
        rates[(lr_values.index(lr)*100):((lr_values.index(lr)+1)*100)] = rates_lr

    
    

    # Build dataset -> shape (len(lr_values)*100, 20, 1, 200, 200)
    dataset = PhaseDiffDataset(rates, num_timepoints=20)
    
    # Wrap in DataLoader
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    # Instantiate model
    model = ConvLSTMAutoEncoder(input_channels=1, hidden_channels=8)  # smaller hidden_channels to save memory
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    # Training loop
    for epoch in range(num_epochs):
        total_loss = 0.0
        for i, batch in enumerate(loader):
            # batch: (B, 20, 1, 200, 200)
            batch = batch.to(device)
            optimizer.zero_grad()
            
            recon = model(batch)    # shape (B, 20, 1, 200, 200)
            loss = criterion(recon, batch)
            
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            if i % 5 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(loader)}], Loss: {loss.item():.4f}")
        
        avg_loss = total_loss / len(loader)
        print(f"=== End of Epoch {epoch+1}, Avg Loss: {avg_loss:.4f} ===")


Epoch [1/2], Step [1/300], Loss: 1.1209
Epoch [1/2], Step [6/300], Loss: 1379486464.0000
Epoch [1/2], Step [11/300], Loss: 953749760.0000
Epoch [1/2], Step [16/300], Loss: 1178120960.0000
Epoch [1/2], Step [21/300], Loss: 144732128.0000
Epoch [1/2], Step [26/300], Loss: 1223536384.0000
Epoch [1/2], Step [31/300], Loss: 201085760.0000
Epoch [1/2], Step [36/300], Loss: 151853952.0000
Epoch [1/2], Step [41/300], Loss: 123347840.0000
Epoch [1/2], Step [46/300], Loss: 40846080.0000
Epoch [1/2], Step [51/300], Loss: 122252744.0000
Epoch [1/2], Step [56/300], Loss: 22803616.0000
Epoch [1/2], Step [61/300], Loss: 25785512.0000
Epoch [1/2], Step [66/300], Loss: 25566464.0000
Epoch [1/2], Step [71/300], Loss: 17588518.0000
Epoch [1/2], Step [76/300], Loss: 17735226.0000
Epoch [1/2], Step [81/300], Loss: 8877123.0000
Epoch [1/2], Step [86/300], Loss: 7568984.5000
Epoch [1/2], Step [91/300], Loss: 17894426.0000
Epoch [1/2], Step [96/300], Loss: 10067254.0000
Epoch [1/2], Step [101/300], Loss: 2179

In [5]:
# Save the trained model
torch.save(model.state_dict(), "./Results/convlstm_autoencoder.pth")

In [10]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Suppose you have these modules from your previous code
# from your_module import ConvLSTMAutoEncoder, PhaseDiffDataset, project_to_latent, visualize_latent_space


# 1) Suppose we have a trained model (just re-instantiate a model here for demonstration)
model = ConvLSTMAutoEncoder(input_channels=1, hidden_channels=8)
# You would typically do: model.load_state_dict(torch.load("my_trained_model.pt"))
model.eval()

# 2) Create or load your new data batch. 
#    load simulations with lr 500 and 25000
rates_500 = np.load("./Results/homogeneous/g_3.5_lr_500.npy")
rates_25000 = np.load("./Results/homogeneous/g_3.5_lr_25000.npy")
new_data = np.concatenate([rates_500, rates_25000], axis=0)
test_data = PhaseDiffDataset(new_data, num_timepoints=20)
#test_data = test_data[:, :, None, :, :]  # add channel dimension


# 3) We can feed the entire 50 items in one go or in smaller batches. 
#    Let's do it in smaller batches to avoid running out of memory:
batch_size = 5
loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [13]:
new_data = np.concatenate([rates_500[:20], rates_25000[:20]], axis=0)
test_data = PhaseDiffDataset(new_data, num_timepoints=20)
#test_data = test_data[:, :, None, :, :]  # add channel dimension


# 3) We can feed the entire 50 items in one go or in smaller batches. 
#    Let's do it in smaller batches to avoid running out of memory:
batch_size = 5
loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [14]:

latent_all = []
for batch in loader:
    # batch: shape (B, 20, 1, 200, 200)
    with torch.no_grad():
        h_enc, c_enc = model.encoder_lstm(batch)
        # shape => (B, hidden_channels, 200, 200)
        B, hidden_dim, H, W = h_enc.shape
        
        # Flatten
        latent_flat = h_enc.view(B, -1)  # shape => (B, hidden_dim * H * W)
        latent_all.append(latent_flat)

# 4) Combine all batches into one big tensor
latent_all = torch.cat(latent_all, dim=0)  # shape => (50, hidden_dim * H * W)

print("Latent representation shape:", latent_all.shape)

Latent representation shape: torch.Size([40, 320000])


In [15]:
latent_all.shape

torch.Size([40, 320000])

In [ ]:


# 5) Optionally apply t-SNE for 2D visualization
X = latent_all.cpu().numpy()
tsne = TSNE(n_components=2, random_state=42)
X_emb = tsne.fit_transform(X)

# 6) Plot the 2D embedding
plt.figure(figsize=(6,5))
plt.scatter(X_emb[:, 0], X_emb[:, 1], c='blue', alpha=0.7)
plt.title("t-SNE of ConvLSTM Latent Space")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()


Latent representation shape: torch.Size([200, 320000])


KeyboardInterrupt: 